In [38]:
import geopandas as gpd
import importlib as imp
import functions as f
imp.reload(f)
from functions import H3_INDEX_RESOLUTION, CRS, drop_duplicated_rows, keep_only_geo_objects, drop_duplicated_rows,objects_in_geodataframe
imp.reload(f)

<module 'functions' from 'd:\\ITU\\Geospatial Data Science\\PROJECT\\functions.py'>

In [ ]:
# forest_all_wgs84_only_polygons_2 = gpd.read_parquet('dataset/processed/forest_areas_with_bikelanes_wgs84_TEMP.parquet')
bikelane_all_wgs84_h3_indexed = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_WGS84_h3_indexed.parquet') 
# keep only the geometry colummn
# forest_all_wgs84_only_polygons_2 = forest_all_wgs84_only_polygons_2[['geometry']]


###### 

forest_all_wgs84 = gpd.read_parquet('./dataset/raw_unprocessed/denmark_landuse_forest_crs_wgs84.parquet')

### The function

In [57]:
def get_only_areas_which_are_crossed_by_bikelane(gdf_to_keep, bikelanes):
    if gdf_to_keep.crs != bikelanes.crs:
        raise Exception("CRS mismatch. Please make sure both geodataframes have the same CRS.")
    all_geometry_objects = objects_in_geodataframe(gdf_to_keep)
    # drop indexies 
    gdf_to_keep = gdf_to_keep.reset_index(drop=True)
    bikelanes = bikelanes.reset_index(drop=True)
    # keep only geometry columns
    gdf_to_keep = gdf_to_keep[["geometry"]]
    bikelanes = bikelanes[["geometry"]]
    if "Point" in all_geometry_objects or "LineString" in all_geometry_objects:
        gdf_to_keep = keep_only_geo_objects(gdf_to_keep)
    if "MultiPolygon" in all_geometry_objects:
        gdf_to_keep = gdf_to_keep.explode()

    if gdf_to_keep.sindex is None or bikelanes.sindex is None:
        raise Exception("Spatial index missing. Please create a spatial index for both geodataframes.")

    joined_df = gpd.sjoin(
        gdf_to_keep,
        bikelanes,
        how="left", # options are left, right, inner or outer
        predicate="intersects", # can be contains, crosses, overlaps, within, etc.
    )
    print(joined_df.head())
    joined_df.dropna(subset=['index_right'], inplace=True)
    joined_df.drop(columns=['index_right'], inplace=True)
    joined_df = drop_duplicated_rows(joined_df)
    joined_df = joined_df[["geometry"]]

    return joined_df

t = get_only_areas_which_are_crossed_by_bikelane(forest_all_wgs84, bikelane_all_wgs84_h3_indexed)

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_24884\33126857.py:12: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_to_keep = gdf_to_keep.explode()


                                              geometry  index_right
0 0  POLYGON ((14.78364 55.07732, 14.78309 55.07716...          NaN
1 0  POLYGON ((14.79076 55.07818, 14.79083 55.07817...          NaN
2 0  POLYGON ((14.78475 55.08093, 14.78753 55.08021...          NaN
3 0  POLYGON ((14.78204 55.07241, 14.78222 55.07250...          NaN
4 0  POLYGON ((14.79104 55.07503, 14.79077 55.07501...          NaN


In [58]:
t.to_parquet("dataset/processed/forest_areas_crossed_by_bikelane_wgs84.parquet")

### The rest is the mess

In [4]:
f.print_gdf_details(forest_all_wgs84)

GeoDataFrame details:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 212495 entries, 0 to 212494
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  212495 non-null  geometry
dtypes: geometry(1)
memory usage: 1.6 MB
None
Number of duplicated rows: 5742
Memory usage (bytes): 1700092
Memory usage (MB): 1.62 MB
Set of geometry objects: {'Point', 'Polygon', 'MultiPolygon', 'LineString'}


In [8]:
import geopandas as gpd
from shapely.geometry import box

def filter_gdf_by_box(gdf, min_lon, min_lat, max_lon, max_lat):
    """
    Filter a GeoDataFrame based on a bounding box defined by minimum and maximum longitude and latitude.
    :param gdf: GeoDataFrame
    :param min_lon: Minimum longitude
    :param min_lat: Minimum latitude
    :param max_lon: Maximum longitude
    :param max_lat: Maximum latitude
    :return: Filtered GeoDataFrame
    """
    bbox = box(min_lon, min_lat, max_lon, max_lat)
    filtered_gdf = gdf[gdf.intersects(bbox)]
    return filtered_gdf


In [23]:
# filter geodataframe based on box 
box_forest = filter_gdf_by_box(forest_all_wgs84_no_duplication_only_polygons, 12.5, 55.5, 15.5, 57.5)
f.print_gdf_details(box_forest)
box_bike = filter_gdf_by_box(bikelane_all_wgs84_h3_indexed, 12.5, 55.5, 15.5, 57.5)
box_forest = box_forest.explode()

GeoDataFrame details:
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 844 entries, 186736 to 194293
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  844 non-null    geometry
dtypes: geometry(1)
memory usage: 13.2 KB
None
Number of duplicated rows: 0
Memory usage (bytes): 13504
Memory usage (MB): 0.01 MB
Set of geometry objects: {'Polygon', 'MultiPolygon'}


C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_24884\958964810.py:5: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  box_forest = box_forest.explode()


In [24]:
f.print_gdf_details(box_forest)

GeoDataFrame details:
<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 860 entries, (186736, 0) to (194293, 0)
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  860 non-null    geometry
dtypes: geometry(1)
memory usage: 48.4 KB
None
Number of duplicated rows: 0
Memory usage (bytes): 49520
Memory usage (MB): 0.05 MB
Set of geometry objects: {'Polygon'}


In [25]:
box_bike.drop(columns=['h3_index'], inplace=True)
box_bike.reset_index(drop=True, inplace=True)
f.print_gdf_details(box_bike)

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_24884\3995784108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  box_bike.drop(columns=['h3_index'], inplace=True)


GeoDataFrame details:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 111656 entries, 0 to 111655
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  111656 non-null  geometry
dtypes: geometry(1)
memory usage: 872.4 KB
None
Number of duplicated rows: 0
Memory usage (bytes): 893380
Memory usage (MB): 0.85 MB
Set of geometry objects: {'LineString'}


In [27]:
%%time

joined_trees_inter = gpd.sjoin(
        box_forest,
        box_bike,
        how="left", # options are left, right, inner or outer
        predicate="intersects", # can be contains, crosses, overlaps, within, etc.
    )

CPU times: total: 62.5 ms
Wall time: 83 ms


In [28]:
# delete rows where index_right is NaN
joined_trees_inter.dropna(subset=['index_right'], inplace=True)

In [29]:
joined_trees_inter.drop(columns=['index_right'], inplace=True)

In [32]:
joined_trees_inter = drop_duplicated_rows(joined_trees_inter)

In [33]:
f.print_gdf_details(joined_trees_inter)

GeoDataFrame details:
<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 267 entries, (186805, 0) to (194293, 0)
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  267 non-null    geometry
dtypes: geometry(1)
memory usage: 42.0 KB
None
Number of duplicated rows: 0
Memory usage (bytes): 42997
Memory usage (MB): 0.04 MB
Set of geometry objects: {'Polygon'}


In [48]:
%%time

joined_trees_inter = gpd.sjoin(
        forest_all_wgs84_no_duplication_only_polygons,
        bikelane_all_wgs84_h3_indexed,
        how="left", # options are left, right, inner or outer
        predicate="intersects", # can be contains, crosses, overlaps, within, etc.
    )

CPU times: total: 2.73 s
Wall time: 6.88 s


In [49]:
# keep only the geometry colummn
joined_trees_inter = joined_trees_inter[['geometry']]

In [50]:
f.print_gdf_details(joined_trees_inter)

GeoDataFrame details:
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 523416 entries, 0 to 212494
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  523416 non-null  geometry
dtypes: geometry(1)
memory usage: 8.0 MB
None
Number of duplicated rows: 330030
Memory usage (bytes): 8374656
Memory usage (MB): 7.99 MB
Set of geometry objects: {'Polygon', 'MultiPolygon'}


In [51]:
joined_trees_inter = joined_trees_inter.drop_duplicates(subset=['geometry'])

In [52]:
f.print_gdf_details(joined_trees_inter)

GeoDataFrame details:
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 193386 entries, 0 to 212494
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  193386 non-null  geometry
dtypes: geometry(1)
memory usage: 3.0 MB
None
Number of duplicated rows: 0
Memory usage (bytes): 3094176
Memory usage (MB): 2.95 MB
Set of geometry objects: {'Polygon', 'MultiPolygon'}
